In [1]:
import os
import pandas as pd
import random
import numpy as np
from scipy.signal import find_peaks

# データが保存されているパスと保存先のパスを設定
base_data_path = r"C:\Users\r-fujita\Desktop\Phase2\教師データExcel"
output_folder = r"C:\Users\r-fujita\Desktop\Phase2\dataset_normal"
analysis_data_path = r"C:\Users\r-fujita\Desktop\Phase2\教師データJUDGE"

# 解析データの一覧を取得
analysis_files = os.listdir(analysis_data_path)
analysis_files = [file for file in analysis_files if file.endswith('.xlsx')]

def detect_peaks(data, prominence=1, distance=50):
    # オフセットを0近辺に調整
    data = data - np.mean(data)
    
    peaks, _ = find_peaks(data, prominence=prominence, distance=distance)
    return peaks

def save_selected_points(peaks, base_filename, original_filename):
    for peak_index in peaks:
        start = max(0, peak_index - 100)  # 幅200の半分である100点前
        end = min(data_length, peak_index + 100)  # 幅200の半分である100点後
        
        # 出力ファイル名
        output_filename = f"{base_filename}_{original_filename}_start_{start}_end_{end}.xlsx"
        output_path = os.path.join(output_folder, output_filename)
        
        # 選択した行を新しいExcelファイルに保存
        window_data = data.iloc[start:end+1]  # 終了位置を含めるために+1
        window_data.to_excel(output_path, index=False, header=False)
        print(f"Saved {output_filename}")

# 各サブフォルダから20ファイルずつランダムに選択
# random_analysis_files = select_random_files(analysis_data_path)

# 各解析データについてループ処理
# processed_file_count = 0
for analysis_file in analysis_files:
    # if processed_file_count >= 500:
    #     break

    # 解析データを読み込む
    analysis_data = pd.read_excel(os.path.join(analysis_data_path, analysis_file), header=None)
    
    # base_filenameとoriginal_filenameを初期化
    base_filename = os.path.splitext(analysis_file)[0]
    
    # 各行の処理
    for index, row in analysis_data.iterrows():
        # if processed_file_count >= 500:
        #     break

        # 管番号の生成
        col1 = str(row[0]).zfill(3)  # 1列目の数値
        col2 = str(row[1]).zfill(3)  # 2列目の数値
        tube_number = f"{col1}-{col2}"
        
        col4 = row[3]
        
        # 1行目の4列目の値が0である場合の処理
        if col4 == 0:
            # データに対応するフォルダとファイル名を作成
            folder_name = analysis_file.split('.')[0]
            data_folder_path = os.path.join(base_data_path, folder_name)
            data_file_path = os.path.join(data_folder_path, f"{tube_number}.xlsx")
            
            # 該当するデータファイルを読み込む
            try:
                data = pd.read_excel(data_file_path)
                data_length = len(data)
                
                if data_length <= 100:
                    continue  # データの長さが100以下の場合はスキップ
                
                # データ全体を取得
                subset_data = data.iloc[:, 6]  # 7番目のカラムを使用
                
                # データの両端15％をカット
                start_index = int(data_length * 0.15)
                end_index = int(data_length * 0.85)
                subset_data = subset_data.iloc[start_index:end_index]

                # 高振幅部上位3点を抽出
                top_peaks = np.argsort(subset_data.values)[-1:] + start_index

                # 重複しないように、抽出範囲を検索範囲から除外
                for peak_index in top_peaks:
                    start = max(0, peak_index - 100)
                    end = min(data_length, peak_index + 100)
                    subset_data = subset_data.iloc[start:end]

                # ピーク位置を中心に前後合わせて100点のデータを保存
                original_filename = os.path.basename(data_file_path)
                save_selected_points(top_peaks, base_filename, original_filename)

                # 処理したファイルカウントを更新
                # processed_file_count += 1
            
            except Exception as e:
                print(f"Error processing {tube_number}: {str(e)}")

print("Processing complete.")

Saved BCU_E-5432_001-001.xlsx_start_3218_end_3418.xlsx
Saved BCU_E-5432_001-002.xlsx_start_7981_end_8181.xlsx
Saved BCU_E-5432_001-003.xlsx_start_7601_end_7801.xlsx
Saved BCU_E-5432_001-004.xlsx_start_5739_end_5939.xlsx
Saved BCU_E-5432_001-005.xlsx_start_7204_end_7404.xlsx
Saved BCU_E-5432_001-006.xlsx_start_5897_end_6097.xlsx
Saved BCU_E-5432_002-001.xlsx_start_6824_end_7024.xlsx
Saved BCU_E-5432_002-002.xlsx_start_2159_end_2359.xlsx
Saved BCU_E-5432_002-003.xlsx_start_8207_end_8407.xlsx
Saved BCU_E-5432_002-004.xlsx_start_6385_end_6585.xlsx
Saved BCU_E-5432_002-005.xlsx_start_1937_end_2137.xlsx
Saved BCU_E-5432_002-006.xlsx_start_2187_end_2387.xlsx
Saved BCU_E-5432_002-007.xlsx_start_3021_end_3221.xlsx
Saved BCU_E-5432_003-001.xlsx_start_2014_end_2214.xlsx
Saved BCU_E-5432_003-002.xlsx_start_9267_end_9467.xlsx
Saved BCU_E-5432_003-003.xlsx_start_1599_end_1799.xlsx
Saved BCU_E-5432_003-004.xlsx_start_8147_end_8347.xlsx
Saved BCU_E-5432_003-005.xlsx_start_8200_end_8400.xlsx
Saved BCU_

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [2]:
# ヘッダーを削除するExcelファイルが格納されているフォルダのパス
folder_path = r"C:\Users\r-fujita\Desktop\Phase2\assets"

# フォルダ内のすべてのExcelファイルを取得
excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# 各Excelファイルに対してヘッダーを削除
for excel_file in excel_files:
    # ファイルの完全パスを取得
    file_path = os.path.join(folder_path, excel_file)

    # Excelファイルを読み込む
    df = pd.read_excel(file_path)

    # ヘッダーを削除して新しいExcelファイルに保存
    output_file_path = os.path.join(folder_path, f"{os.path.splitext(excel_file)[0]}_noheader.xlsx")
    df.to_excel(output_file_path, index=False, header=False)

    print(f"ヘッダーを削除しました: {excel_file}")

ヘッダーを削除しました: BCU_E-5432_001-001.xlsx_start_3218_end_3418.xlsx
ヘッダーを削除しました: BCU_E-5432_001-002.xlsx_start_7981_end_8181.xlsx
ヘッダーを削除しました: BCU_E-5432_001-003.xlsx_start_7601_end_7801.xlsx
ヘッダーを削除しました: BCU_E-5432_001-004.xlsx_start_5739_end_5939.xlsx
ヘッダーを削除しました: BCU_E-5432_001-005.xlsx_start_7204_end_7404.xlsx
ヘッダーを削除しました: BCU_E-5432_001-006.xlsx_start_5897_end_6097.xlsx
ヘッダーを削除しました: BCU_E-5432_002-001.xlsx_start_6824_end_7024.xlsx
ヘッダーを削除しました: BCU_E-5432_002-002.xlsx_start_2159_end_2359.xlsx
ヘッダーを削除しました: BCU_E-5432_002-003.xlsx_start_8207_end_8407.xlsx
ヘッダーを削除しました: BCU_E-5432_002-004.xlsx_start_6385_end_6585.xlsx
ヘッダーを削除しました: BCU_E-5432_002-005.xlsx_start_1937_end_2137.xlsx
ヘッダーを削除しました: BCU_E-5432_002-006.xlsx_start_2187_end_2387.xlsx
ヘッダーを削除しました: BCU_E-5432_002-007.xlsx_start_3021_end_3221.xlsx
ヘッダーを削除しました: BCU_E-5432_003-001.xlsx_start_2014_end_2214.xlsx
ヘッダーを削除しました: BCU_E-5432_003-002.xlsx_start_9267_end_9467.xlsx
ヘッダーを削除しました: BCU_E-5432_003-003.xlsx_start_1599_end_1799.xlsx
ヘッダーを削除し